In [2]:
import pandas as pd
import numpy as np

In [4]:
count=0
for chunk in pd.read_csv("nurseCharting.csv", chunksize=10000, usecols=[0]):
    count+=1
print(count)

15161


In [6]:
print("rows",15161*10000)

rows 151610000


151 millions rows in nurseCharting, chunking is inevitable

In [7]:
l_nursechart=[]
for chunk in pd.read_csv("nurseCharting.csv", chunksize=10000, usecols=[1, 3, 4, 5, 7]):
    df=chunk.loc[(chunk['nursingchartcelltypecat']=='Scores') & (chunk['nursingchartcelltypevallabel']=='Glasgow coma score')]
    l_nursechart.append(df)

In [8]:
df_nursechart=pd.concat(l_nursechart, sort=False)

In [9]:
df_nursechartrt.head()

,patientunitstayid,nursingchartentryoffset,nursingchartcelltypecat,nursingchartcelltypevallabel,nursingchartvalue
300,141924,2109,Scores,Glasgow coma score,15
480,141924,1894,Scores,Glasgow coma score,15
636,141924,424,Scores,Glasgow coma score,15
991,141944,484,Scores,Glasgow coma score,15
1022,141944,2389,Scores,Glasgow coma score,14


In [10]:
df_nursechart.shape

(11497735, 5)

In [11]:
import csv

In [12]:
df_nursechart.to_csv("gcs_scores.csv",sep=',',index=False,encoding='utf-8')

In [3]:
df_gcs=pd.read_csv("gcs_scores.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_gcs.head()

,patientunitstayid,nursingchartentryoffset,nursingchartcelltypecat,nursingchartcelltypevallabel,nursingchartvalue
0,141924,2109,Scores,Glasgow coma score,15
1,141924,1894,Scores,Glasgow coma score,15
2,141924,424,Scores,Glasgow coma score,15
3,141944,484,Scores,Glasgow coma score,15
4,141944,2389,Scores,Glasgow coma score,14


<h4>Make a column only of GCS scores</h4>

In [5]:
df_gcs['GCS_Score']=np.nan

In [6]:
df_gcs.dtypes

patientunitstayid                 int64
nursingchartentryoffset           int64
nursingchartcelltypecat          object
nursingchartcelltypevallabel     object
nursingchartvalue                object
GCS_Score                       float64
dtype: object

In [7]:
df_gcs['nursingchartvalue']=df_gcs['nursingchartvalue'].astype('float64')

ValueError: could not convert string to float: 'Unable to score due to medication'

If needed<br>
df_gcs['nursingchartvalue']=df_gcs['nursingchartvalue'].ffill()<br>
df_gcs['nursingchartvalue']=df_gcs['nursingchartvalue'].bfill()

Removing all such entries with chartvalue=="Unable to score due to medication"

In [8]:
df_gcs=df_gcs.loc[~df_gcs['nursingchartvalue'].str.contains("unable",case=False,na=False)]

In [9]:
any(df_gcs['nursingchartvalue'].isna())

True

Now, we crosscheck rows where the column <b>nursingchartvalue</b> is throwing float value error. As we can see, we have cleared all the fault cases.

In [11]:
index=0
count=0
for value in df_gcs['nursingchartvalue']:
    index+=1
    try:
        x=float(value)
    except ValueError as e:
        count+=1
        print("error",e,"in line",index)

In [10]:
len(df_gcs)

11419171

In [11]:
df_gcs[['GCS_Score']]=df_gcs[['nursingchartvalue']]

In [19]:
del df_gcs['nursingchartvalue']
df_gcs.to_csv("gcs_scores_updated.csv",sep=',',index=False,encoding='utf-8')

In [12]:
#Adding SOFA Scores
d_gcs=pd.read_csv('gcs_scores_updated.csv')
columns=['patientunitstayid','nursingchartentryoffset','GCS_Score']
d_gcs=d_gcs[columns]
d_gcs=d_gcs.dropna()

d_gcs.loc[d_gcs['GCS_Score']>=15,'SOFA_GCS']=0
d_gcs.loc[d_gcs['GCS_Score']<15,'SOFA_GCS']=1
d_gcs.loc[d_gcs['GCS_Score']<13,'SOFA_GCS']=2
d_gcs.loc[d_gcs['GCS_Score']<10,'SOFA_GCS']=3
d_gcs.loc[d_gcs['GCS_Score']<6,'SOFA_GCS']=4

d_gcs=d_gcs.rename(columns={'nursingchartentryoffset':'offset'})

In [21]:
d_gcs.head()

,patientunitstayid,nursingchartentryoffset,GCS_Score,SOFA_GCS
0,141924,2109,15.0,0.0
1,141924,1894,15.0,0.0
2,141924,424,15.0,0.0
3,141944,484,15.0,0.0
4,141944,2389,14.0,1.0


In [6]:
d_gcs.to_csv("gcs_withSOFA.csv",index=False)

In [13]:
l_labs=[]
for chunk in pd.read_csv("lab.csv",chunksize=10000, usecols=[1, 2, 4, 5, 7, 8, 9]):
    df=chunk.loc[(chunk['labname']=='paO2') | (chunk['labname']=='platelets x 1000') | 
                (chunk['labname']=='total bilirubin') | (chunk['labname']=='urinary creatinine')]
    l_labs.append(df)
df_labs=pd.concat(l_labs, sort=False)


In [14]:
df_labs.head()

,patientunitstayid,labresultoffset,labname,labresult,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset
6,141168,1701,urinary creatinine,173.12,mg/dL,mg/dL,1757
24,141168,2026,paO2,68.00,mm Hg,mm Hg,2054
28,141168,2141,paO2,121.00,mm Hg,mm Hg,2155
29,141168,1133,platelets x 1000,213.00,K/mcL,K/mcL,1196
45,141168,2010,paO2,42.00,mm Hg,mm Hg,2026


In [15]:
df_labs.to_csv("labs_before_FiO2.csv", sep=',', index=False, encoding='utf-8')

In [16]:
df_labs.shape

(2010839, 7)

<h1>Continue from here</h1>

In [2]:
l_map=[]
for chunk in pd.read_csv("nurseCharting.csv", chunksize=10000, usecols=[1, 3, 4, 5, 7]):
    df=chunk.loc[(chunk['nursingchartcelltypevallabel']=='MAP (mmHg)')]
    l_map.append(df)


In [3]:
df_map=pd.concat(l_map,sort=False)

In [4]:
df_map.head()

,patientunitstayid,nursingchartentryoffset,nursingchartcelltypecat,nursingchartcelltypevallabel,nursingchartvalue
4,141924,1489,Other Vital Signs and Infusions,MAP (mmHg),71
38,141924,1224,Other Vital Signs and Infusions,MAP (mmHg),79
67,141924,1564,Other Vital Signs and Infusions,MAP (mmHg),73
69,141924,994,Other Vital Signs and Infusions,MAP (mmHg),96
74,141924,4144,Other Vital Signs and Infusions,MAP (mmHg),78


In [5]:
df_map.to_csv("nursechartMAP.csv",index=False)

In [6]:
l_vent=[]
for chunk in pd.read_csv("nurseCharting.csv", chunksize=10000, usecols=[1, 3, 4, 5, 7]):
    df=chunk.loc[(chunk['nursingchartcelltypevallabel']=='O2 Admin Device')]
    l_vent.append(df)
df_vent=pd.concat(l_vent,sort=False)
df_vent.to_csv("nursechartVent.csv",index=False)